# Frozen Lake
The Frozen Lake environment is an uncertain grid world in which you start from an initial state (the top leftmost square) and move to a final state (the bottom rightmost square). The environment is uncertain because you are walking on a frozen lake and the ice thickness varies. So you can fall into the water in some squares. Also, the ice is more slippery in some places, so taking a step can take you further than expected... and if the wind gets in the way...
Instead of trying to estimate the transition pattern, we'll use SARSA and Q-learning to solve this problem.
Use the Frozen Lake environment to implement SARSA and Q-learning. First use the environment with a 4x4 grid to test your algorithms, then you should be able to use them for the 16x16 grid.

#FrozenLake - familiarization with the environment
Evaluate a random policy.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import gymnasium as gym

env=gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True)

numStates = env.observation_space.n
numActions = env.action_space.n
print("Environnement with ", numStates, " states and ", numActions, " actions")
#
# env.reset() resets and starts a new episode
#
state = env.reset()
nbIt=0
rew=[]
done=False
while not done:
  #
  # env.step(action) executes an action in the current state
  # the method returns
  #    • the next state
  #    • the immediate reward
  #    • a boolean that tells whether the episode has ended
  #    • another argument that is usefull for debugging purposes
  #
  nextState, reward, done, trunc, info = env.step(np.random.randint(4))
  print("state number:",nextState)

  nbIt+=1
  rew = rew+[reward]
print("Episod ended after {} iterations".format(nbIt))
print("Reward obtained:",rew)
env.close()

Environnement with  16  states and  4  actions
state number: 4
state number: 8
state number: 8
state number: 8
state number: 4
state number: 5
Episod ended after 6 iterations
Reward obtained: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
env.reset()

(0, {'prob': 1})

In [ ]:
env.step(0)

(0, 0.0, False, False, {'prob': 0.3333333333333333})

## $\epsilon$-greedy

Implement a function that returns an action with the $\epsilon$-greedy strategy:
* explores with probability $1-\epsilon$: here we choose the action with the best value of $q[s]$
* explores with a probability $\epsilon$: we choose an action in a uniform way on all the actions.

You can choose different signatures for the function:
either by passing it:
 * the parameter $\epsilon$
 * the Q table
 * the s state in which the action will be executed
 * so the call will have the form `action = epsGreedy(eps, Q, s)`

 Alternatively, you can give only the value of $\epsilon$ and vector Q(s) (whose dimension is the number of actions). The call will then have the form `action = epsGreedy(eps, q)`

*Please note* One can imagine the particular case where there are several occurrences of the max value in the vector `Q(s)`. In this case, one should not *always* choose the same action, but rather choose one of the ex-aequo actions at random.
This case may not be so exotic, especially at the beginning of the learning process, when all values are zero. To explore, it is then desirable to repeat the same choice!


For those unfamiliar with python, take a look at the small code example below to illustrate some of the functions in the `numpy` library
- The function `np.random.rand()` draws a value uniformly between 0 and 1.
- The function `np.random.choice` draws a value uniformly from a set.
- The function `np.argwhere(l)` allows to give the indices where the input of the vector l is non-zero. We can therefore couple a call to `np.argwhere` with a test.

In [ ]:
val = np.random.rand()
print(val)
val=np.zeros(10)
# we draw a sample uniformly in the set {1, 3, 5}
for i in range(10):
  val[i]=np.random.choice([1,3,5])
_, count = np.unique(val,return_counts=True)
print("result from the 10 draws:", val)
print("the proportion of each value over these 10 draws are ", count/10)
indices3 = np.argwhere(val==3)
print("the indices of the list val in which the value is 3 are:", indices3)

#def epsGreedy():
   à compléter

0.711624653012464
result from the 10 draws: [3. 1. 5. 1. 3. 3. 3. 5. 5. 3.]
the proportion of each value over these 10 draws are  [0.2 0.5 0.3]
the indices of the list val in which the value is 3 are: [[0]
 [4]
 [5]
 [6]
 [9]]


## Testing a policy

When learning, it is necessary to explore, so when analyzing the performance during learning, one must keep in mind that some of the choices are made at random. After learning, one can test by being gluttonous: in each state, one always chooses the action that gives the highest value of `Q`.

Implement a method that takes as parameter a fixed number of episodes, a `Q` table, and executes the gluttonous policy. The method returns the average value of the sum of the rewards over the episode.

In [ ]:
# implement!

### SARSA

Implement a SARSA function that takes as parameter
 * a number of episodes used for learning
 * $\gamma$ the discount rate
 * $\alpha$ the learning rate (which is found when updating the values of Q)
 * $\epsilon$ the parameter for the $\epsilon$-greedy method.

Your function must at least return the table $Q: S \times A$. You will find below a function $plotQ$ which generates a representation of the table $Q$: for each cell will be drawn the best action according to $Q$ and the color will represent the value of this action.

To visualize the progress made during learning, your SARSA function can also return a sequence of values. For example,
 * the sequence of rewards (total or average) obtained on each learning episode
 * the value of the best action for the starting state at the end of each episode.
 * Instead of using the values obtained during the training, you can also periodically evaluate the current policy (without exploration). To do this, you can calculate the performance over a small number of episodes and return the average. This method has the advantage of evaluating the policy without exploration (thus a better evaluation of the policy), but can be expensive in computation time depending on the frequency of execution and the number of episodes used for the evaluation.

When generating the graph, you should visualize if the algorithm has managed to improve the performance. You can either plot the value of each episode directly. To get a smoother curve, you can also calculate an average over a window of $k$ episodes (the $runningAvg$ function does this job).

Note that Frozen lake is considered solved when
 * it reaches the goal in 78% of the episodes for the 4x4 grid.
 * a priori, we can reach 100% for the 8x8 grid

Some ideas to help with the debug:
 * you can also look at whether most of the state-action pairs have been executed.
 * You can choose as parameters (the code I wrote worked with these parameters, obviously, you can try with others later).
   * $\epsilon=0.2$
   * $\alpha=0.02$
   * Frozen lake is an episodic task, so here we can simply be interested in the sum of the rewards accumulated during an episode. So we can choose $\gamma=1$ (no discount).

In [ ]:
def runningAvg(data, windowSize):
  res = np.zeros(len(data)-windowSize)
  sum=0
  for i in range(windowSize):
    sum += data[i]
  for i in range(len(data)-windowSize):
    res[i]= sum/windowSize
    sum -= data[i]
    sum += data[i+windowSize]
  return res


# visualisation de la table Q pour FrozenLake 4x4 et 8x8
# passez la taille (4 ou 8) en paramètres
def plotQ(q_table, map_size):
  if (map_size==4):
    MAP = [
        "SFFF",
        "FHFH",
        "FFFF",
        "HFFG"
    ]
  else:
    MAP=[
        "SFFFFFFF",
        "FFFFFFFF",
        "FFFHFFFF",
        "FFFFFHFF",
        "FFFHFFFF",
        "FHHFFFHF",
        "FHFFHFHF",
        "FFFHFFFG"
    ]
  best_value = np.max(q_table, axis = 1).reshape((map_size,map_size))
  best_policy = np.argmax(q_table, axis = 1).reshape((map_size,map_size))

  fig, ax = plt.subplots()
  im = ax.imshow(best_value)

  for i in range(best_value.shape[0]):
      for j in range(best_value.shape[1]):
          if MAP[i][j] in 'GH':
              arrow = MAP[i][j]
          elif best_policy[i, j] == 0:
              arrow = '<'
          elif best_policy[i, j] == 1:
              arrow = 'v'
          elif best_policy[i, j] == 2:
              arrow = '>'
          elif best_policy[i, j] == 3:
              arrow = '^'
          if MAP[i][j] in 'S':
              arrow = 'S ' + arrow
          text = ax.text(j, i, arrow, ha = "center", va = "center",
                         color = "black")

  cbar = ax.figure.colorbar(im, ax = ax)

  fig.tight_layout()
  plt.show()

## Q-learning
Implement the Q-learning algorithm (starting from SARSA, there should only be a few lines of code to modify!)

## Comparison

Compare the policies found using SARSA, Q-learning, and you should also be able to use the code for the on policy Monte Carlo algorithm from the previous TD.

Before convergence to the optimal, we often observe that SARSA has chosen a less risky policy before falling on the optimal for FrozenLake8x8.

## Solving a tabular version of Cart-pole

Finally, we propose to use your code and to test the learning on the cart-pole problem. A priori, this is a problem where the states are continuous variables. We propose here to discretize the variables and to try to use one of the methods to see your results.

The reward you get is the number of time steps where the stick stayed in equilibrium. If you use colab to code, you will unfortunately not be able to view an episode with the render method :-(

This Cart-Pole environment involves moving a cart to balance a beam. More precisely:
* There are two actions: left and right (represented by 0 and 1).
* The received observation (i.e. the state) is a numpy array with 4 variables: the position of the cart, the velocity, the angle to the vertical and the position of the top of the beam.
* The episode ends when the angle of the beam to the vertical exceeds 12 degrees.
* The rewards received are equal to 1 unless the angle exceeds 12 degrees.

Below you are given the functions to perform the discretization and to encode the state into an integer.

In [ ]:
env = gym.make("CartPole-v1")
print("environnement with ", env.action_space.n, " actions")
print("the action space is code with a class ", env.observation_space,
      " representing a continuous space")
print("the lower bounds are: ", env.observation_space.low)
print("the upper bounds are ",env.observation_space.high)
env.reset()
nbIt=0
done=False
print(env.step(np.random.randint(2)))
while not done:
  nextState, reward, done, info, _ = env.step(np.random.randint(2))
  nbIt+=1
print("Episode terminated after {} iterations".format(nbIt))
env.close()

environnement with  2  actions
the action space is code with a class  Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)  representing a continuous space
the lower bounds are:  [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
the upper bounds are  [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
(array([-0.024397  , -0.21372429,  0.00895479,  0.25025564], dtype=float32), 1.0, False, False, {})
Episode terminated after 26 iterations


Modify your implementation of Q-learning and/or SARSA to test if you can learn to keep the stick balanced. One modification will be to use the above functions to encode/decode a state. Another modification will probably be to add the number of states as a parameter because this number is now independent of the environment!
With $\epsilon=0.1$, $\alpha=0.2$ and $\gamma=0.9$ as parameters, I can reach a score around 90 time steps.

In [ ]:
def discretise(x,mini,maxi):
  # discretise x

  if x<mini: x=mini
  if x>maxi: x=maxi
  return int(np.floor((x-mini)*nval/(maxi-mini+0.0001)))

#def encode(observation):
#  pos = discretise(observation[0],mini=-4.8,maxi=4.8)
#  vel = discretise(observation[1],mini=-10,maxi=10)
#  angle = discretise(observation[2],mini=-0.42,maxi=0.42)
#  pos2 = discretise(observation[3],mini=-1,maxi=1)
#  return pos + vel*nval + angle*nval*nval + pos2*nval*nval*nval

def encode(observation):
  pos = discretise(observation[0],mini=-1,maxi=1)
  vel = discretise(observation[1],mini=-1,maxi=1)
  angle = discretise(observation[2],mini=-1,maxi=1)
  pos2 = discretise(observation[3],mini=-1,maxi=1)
  return pos + vel*nval + angle*nval*nval + pos2*nval*nval*nval

nval =5 # number of discrete values that a continuous variable can take (granularity)
N= nval ** 4 # state space size
print("The number of states is ", N)


The number of states is  625
